# Introduction

# Setup
Let us import the required modules.

In [1]:
import pandas as pd
import seaborn as sns
import os
import joblib

import src.data_split as ds
import src.evaluation as eva

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

## Load Data
Note that the dataset is already split into Train-Test sets.

In [2]:
data_split = ds.TrainTestSplit.from_csv_directory(dir_path="../data")

In [3]:
data_split.x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Columns: 719 entries, pixel13 to pixel780
dtypes: float64(719)
memory usage: 329.1 MB


In [4]:
data_split.y_train.shape

(60000,)

# Support Vector Machine Classifier Model
## LINEAR
SVM  using linear, polynomial of degree 2, and RBF kernels;
LinearSVC is another (faster) implementation of Support Vector Classification for the case of a linear kernel

In [5]:
clf = SVC()
clf.fit(data_split.x_train, data_split.y_train)

SVC()

In [6]:
rf_train_pred = clf.predict(data_split.x_train)
rf_test_pred = clf.predict(data_split.x_test)

rf_train_eval = eva.Evaluation(y_real=data_split.y_train, y_pred=rf_train_pred)
rf_test_eval = eva.Evaluation(y_real=data_split.y_test, y_pred=rf_test_pred)

KeyboardInterrupt: 

In [ ]:
print("Training:")
rf_train_eval.print_eval()
print("Testing:")
rf_test_eval.print_eval()

In [ ]:
# fai lineare, non SVC e basta!!!!!

In [ ]:
import time

# automatic parameters tuning
svcclsf_lin=SVC(random_state=28)
properties={
    "n_estimators": [x for x in range(50,201,50)],
    "min_samples_leaf": [x for x in range(50,201,50)],
    "max_leaf_nodes": [x for x in range(75,176,25)]
}

start_time = time.time()
tuned_svcclsf_lin=GridSearchCV(svcclsf_lin,properties,scoring="accuracy",cv=10,return_train_score=True,verbose=6,n_jobs=4)
tuned_svcclsf_lin.fit(data_split.x_train,data_split.y_train)
print("--- %s seconds ---" % (time.time() - start_time))

print ("Best Score: {:.3f}".format(tuned_svcclsf_lin.best_score_) )
print ("Best Params: ", tuned_svcclsf_lin.best_params_)

In [ ]:
# save results
if not os.path.exists('../tuned_models'):
    os.mkdir('../tuned_models')
joblib.dump(tuned_svcclsf_lin, '../tuned_models/svc_linear_tuning_results.pkl')

In [ ]:
# to load previously saved results
tuned_svcclsf_lin = joblib.load("../tuned_models/svc_linear_tuning_results.pkl")

In [ ]:
print(tuned_svcclsf_lin.cv_results_)
tuned_svcclsf_lin_results = pd.DataFrame( tuned_svcclsf_lin.cv_results_ )

In [ ]:
tuned_svcclsf_lin_results[["param_n_estimators","mean_test_score","mean_train_score"]]

In [ ]:
sns.set(rc={"figure.figsize":(12, 8)})
sns.lineplot(data=tuned_svcclsf_lin_results, x="param_n_estimators", y="mean_test_score")

In [ ]:
print(tuned_svcclsf_lin.best_estimator_)
print(tuned_svcclsf_lin.best_score_)

In [ ]:
test_acc = accuracy_score(y_true = data_split.y_train,
                          y_pred = tuned_svcclsf_lin.predict(data_split.x_train) )
print ("Train Accuracy: {}".format(test_acc) )

In [ ]:
# best parameters from automatic parameters tuning
svc_lin_clsf = SVC(**tuned_svcclsf_lin.best_params_)
# svc_lin_clsf = SVC(max_leaf_nodes=100, min_samples_leaf=75, n_estimators=200, random_state=28)

svc_lin_clsf.fit(data_split.x_train, data_split.y_train)

In [ ]:
svc_lin_train_pred = svc_lin_clsf.predict(data_split.x_train)
svc_lin_test_pred = svc_lin_clsf.predict(data_split.x_test)

svc_lin_train_eval = eva.Evaluation(y_real=data_split.y_train, y_pred=rf_train_pred)
svc_lin_test_eval = eva.Evaluation(y_real=data_split.y_test, y_pred=rf_test_pred)

In [ ]:
print("Training:")
svc_lin_train_eval.print_eval()
print("Testing:")
svc_lin_test_eval.print_eval()

In [ ]:
# rf_model = eva.EvaluatedModel(forest_clsf, train_eval=svc_lin_train_eval, test_eval=rf_test_eval)

accuracy is not a great measure of classifier performance when the classes are imbalanced
but as we can see from the plots, the classes are more or less balanced

# Performance Visualization

### Setup